In [ ]:
import torchaudio
import torch 
import pandas as pd
import numpy as np
import re 
import os 
from torch.nn import functional as F
import random
import math
import pickle
import gc
import torch.nn as nn
from tqdm.notebook import tqdm
import os
import glob
import warnings
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings(action='ignore')

In [ ]:
base_dir = '/home/work/'
np1 = glob.glob(base_dir+'mel_dataset1/*.npy')
np2 = glob.glob(base_dir+'mel_dataset2/*.npy')
np3 = glob.glob(base_dir+'mel_dataset3/*.npy')
np4 = glob.glob(base_dir+'mel_dataset4/*.npy')
np5 = glob.glob(base_dir+'mel_dataset5/*.npy')
np6 = glob.glob(base_dir+'mel_dataset6/*.npy')
np7 = glob.glob(base_dir+'mel_dataset7/*.npy')
np8 = glob.glob(base_dir+'mel_dataset8/*.npy')

In [ ]:
total = np1+np2+np3+np4+np5+np6+np7+np8

In [ ]:
train_csv = pd.read_csv('new_train.csv')
valid_csv = pd.read_csv('new_valid.csv')
all_csv =pd.read_csv('new_all.csv')

In [ ]:
def npy_list(csv):
    npy_list = []
    for song_path in tqdm(csv['npy_path']):
        song_npy = np.load(song_path).squeeze()
        #npy_list.append(song_npy[-1876:])
        npy_list.append(song_npy)
        
    return npy_list

train_list = [np.load(song_path).squeeze() for song_path in tqdm(train_csv['npy_path'])]
valid_list = [np.load(song_path).squeeze() for song_path in tqdm(valid_csv['npy_path'])]


  0%|          | 0/3825 [00:00<?, ?it/s]

  0%|          | 0/675 [00:00<?, ?it/s]

In [ ]:
all_list = train_list + valid_list

In [ ]:

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)


cuda


In [ ]:
def train(model, train_loader):
    model.train()
    train_loss = 0
    for batch in train_loader: 
        
        
        mel = torch.FloatTensor(batch).to(DEVICE)
    
        optimizer.zero_grad()
        
        encode, output = model(mel)
        
        loss = criterion(output, mel)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
    train_loss /= (len(train_loader))

    return train_loss

def val(model, train_loader):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in train_loader:
            
            
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)

            loss = criterion(output, mel)
            
            val_loss += loss.item()

    val_loss /= (len(train_loader))

    return val_loss


def get_mel_embeding(model, train_loader):
    model.eval()
    mel_embeding_li = []
    with torch.no_grad():
        for batch in train_loader: 
            
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)
            mel_embeding_li.append(encode.detach().cpu().numpy())

    return mel_embeding_li

In [ ]:
#charlie
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
    
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )
            
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 256),
            #nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(256, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 1024, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 64)
        )

        
        
        
        self.conv1to3 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
        
        self.conv2to4 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
        
        self.conv3to5 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
        
        self.conv4to6 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
        
        self.conv5to7 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
        
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x1 = self.conv1(x) 
        
        x1to2 = F.pad(x1, pad = (4, 0, 0, 0)) 
        x1to3 = F.pad(x1, pad = (8, 0, 0, 0)) 
        
        
        x2 = self.conv2(x1to2) 
        
        x2to3 = F.pad(x2, pad = (8, 0, 0, 0)) 
        
        x3 = self.conv3(x2to3) 
        x3_connec = self.conv1to3(x1to3) 
        x3= torch.cat([x3,x3_connec],1) 
        
        
        x3to4 = F.pad(x3, pad = (16, 0, 0, 0))
        x3to5 = F.pad(x3, pad = (32, 0, 0, 0)) 
        
        x4 = self.conv4(x3to4) 
        
        x4to5 = F.pad(x4, pad = (32, 0, 0, 0)) 
        

        x5 = self.conv5(x4to5)  
        x5_connec = self.conv3to5(x3to5) 
        x5 = torch.cat([x5,x5_connec],1) 
        
        x5to6 = F.pad(x5, pad = (64, 0, 0, 0)) 
        x5to7 = F.pad(x5, pad = (128, 0, 0, 0)) 
        
        
        x6 = self.conv6(x5to6) 
        x6to7 = F.pad(x6, pad = (128, 0, 0, 0)) 
        
        x7 = self.conv7(x6to7) 
        pre_encode = torch.flatten(x7)
        x7_connec = self.conv5to7(x5to7) 
        x7 = torch.cat([x7,x7_connec],1) 
        
        encode = self.encoder_fc(x7.view(-1, 8 * 7501))

        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        
        
        x = torch.cat([x7,x],1) 
        x = F.pad(x, pad = (2, 0, 0, 0)) 
        x = self.t_conv1(x) 
        
        x = torch.cat([x6,x],1)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv2(x)
        
        
        x = torch.cat([x5,x],1)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv3(x)
        
        
        
        x = torch.cat([x4,x],1)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        
        
        x = torch.cat([x3,x],1)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv5(x)
        
        
        
        x = torch.cat([x2,x],1)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv6(x)
        
        
        
        x = torch.cat([x1,x],1)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.t_conv7(x)
        
        return encode, x


In [ ]:
model = TimeAutoEncoder().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.MSELoss()

In [ ]:
gc.collect()

26

In [ ]:

import time

model_dir = '/home/work/Tcae_apply/model_dir3/'
min_loss = 987654321

epochs = 150
batch_size = 32

train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=False)
val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=False)

for epoch in tqdm(range(1, epochs + 1)):
    start = time.time()
    
    
    train_loss = train(model = model, train_loader = train_batch_li) 
    val_loss = val(model = model, train_loader = val_batch_li) 
    end = time.time()
    
    print(f'EPOCH:{epoch}, Train Loss:{train_loss}, Val Loss:{val_loss}, 학습 시간: {end - start}')
    if (val_loss < min_loss) & (val_loss < 10) & (train_loss < 15) :
        min_loss = val_loss
        torch.save(model.state_dict(), model_dir + f'TimeAutoEncoder_skipconnection_charlie_val.pt')
        print('모델 저장')
    

  0%|          | 0/150 [00:00<?, ?it/s]

EPOCH:1, Train Loss:110260.32459309897, Val Loss:11433.421253551136, 학습 시간: 78.66790819168091
EPOCH:2, Train Loss:7394.33687133789, Val Loss:3800.166681463068, 학습 시간: 71.63565707206726
EPOCH:3, Train Loss:2928.3070871988934, Val Loss:1872.6620871803977, 학습 시간: 57.03754138946533
EPOCH:4, Train Loss:1687.3794443766276, Val Loss:1260.5794039639559, 학습 시간: 45.16006374359131
EPOCH:5, Train Loss:1121.6509796142577, Val Loss:912.6757535067471, 학습 시간: 36.65596675872803
EPOCH:6, Train Loss:864.2644838968913, Val Loss:726.4970619895241, 학습 시간: 35.88951921463013
EPOCH:7, Train Loss:682.7494117736817, Val Loss:561.0816653858532, 학습 시간: 35.71298885345459
EPOCH:8, Train Loss:554.2170783996582, Val Loss:519.9623315984553, 학습 시간: 35.77355408668518
EPOCH:9, Train Loss:607.3109842936198, Val Loss:447.49970176003194, 학습 시간: 35.61007761955261
EPOCH:10, Train Loss:477.72823333740234, Val Loss:532.4633622602983, 학습 시간: 35.18342614173889
EPOCH:11, Train Loss:401.6184939066569, Val Loss:347.6201789162376, 학습 

In [ ]:
model_dir = '/home/work/Tcae_apply/model_dir/'

model = TimeAutoEncoder().to(DEVICE)
model.load_state_dict(torch.load(model_dir + f'TimeAutoEncoder_skipconnection_charlie_val.pt', map_location = DEVICE))

<All keys matched successfully>

In [ ]:
test_batch_li = DataLoader(all_list, batch_size=1, shuffle=False,drop_last=False)


In [ ]:
mel_embeding_li = get_mel_embeding(model = model, train_loader = test_batch_li)
mel_embeding = np.concatenate(mel_embeding_li, axis = 0)

In [ ]:
pd.DataFrame(mel_embeding).to_csv('tcae_charlie_embedding.csv',index=False)

In [ ]:
import glob

In [ ]:
inference_npy_path = '/home/work/Tcae_apply/tcae_inference_folder/mel_folder/'
inference_npy_path = glob.glob(inference_npy_path + '*.npy')

In [ ]:
inference_list = [np.load(song_path).squeeze()[:,:7501] for song_path in tqdm(inference_npy_path)]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
inference_batch_li = DataLoader(inference_list, batch_size=1, shuffle=False,drop_last=False)

In [ ]:
def get_mel_inference_embeding(model, train_loader):
    model.eval()
    mel_embeding_li = []
    with torch.no_grad():
        for batch in train_loader: 
            
            mel = torch.FloatTensor(batch).to(DEVICE)
 
            encode, output = model(mel)
            mel_embeding_li.append(encode.detach().cpu().numpy())

    return mel_embeding_li

In [ ]:
inference_embedding_li = get_mel_embeding(model = model, train_loader = inference_batch_li)
inference_embedding = np.concatenate(inference_embedding_li, axis = 0)

In [ ]:
pd.DataFrame(inference_embedding).to_csv('inference_embedding1.csv',index=False)